In [63]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Concatenate
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model

import os
import shutil
import glob
#from keras import backend as K
#K.set_image_dim_ordering('th')

import numpy as np
import pandas as pd
#import h5py

import matplotlib.pyplot as plt

## 1. Препроцессинг изображений

### 1.1 Очистка папок

In [2]:
def clear_folder(mydir):
    filelist = [f for f in os.listdir(mydir)]
    for f in filelist:
        os.remove(os.path.join(mydir, f))

In [3]:
for i in os.listdir('data/train_a'):
    clear_folder(f'data/train_a/{i}')
    
for i in os.listdir('data/test_a'):
    clear_folder(f'data/test_a/{i}')

for i in os.listdir('data/train_c'):
    clear_folder(f'data/train_c/{i}')
    
for i in os.listdir('data/test_c'):
    clear_folder(f'data/test_c/{i}')

### 1.2 Выбор изображений

In [4]:
data_names_a = list(glob.glob('data/Aldanella_attleborensis/*Plate_?*_?*a.tif'))
data_names_c = list(glob.glob('data/Aldanella_attleborensis/*Plate_?_?*c.tif'))
f_1 = open('data/Aldanella_attleborensis/different_angle.txt', 'rt')
f_2 = open('data/Aldanella_attleborensis/miss_view.txt', 'rt')
diff_angle = f_1.readlines()
miss_view = f_2.readlines()
for i, pic in enumerate(diff_angle):
    diff_angle[i] = f'data/Aldanella_attleborensis\\{pic.rstrip()}.tif'#обрезаем символы на конце строки '\n'
for i, pic in enumerate(miss_view):
    miss_view[i] = f'data/Aldanella_attleborensis\\{pic.rstrip()}.tif'

for pic in diff_angle:
    if pic in data_names_a:
        data_names_a.remove(pic)
        if pic[:-5]+'c.tif' in data_names_c:
            data_names_c.remove(pic[:-5]+'c.tif')
for pic in diff_angle:
    if pic in data_names_c:
        data_names_c.remove(pic)

for pic in miss_view:
    if (pic[-5:-4] == 'c')==True and (pic[:-5]+'a.tif' in diff_angle)==False and (pic[:-5]+'a.tif' in miss_view)==False:
        data_names_a.remove(pic[:-5]+'a.tif')
    if pic[-5:-4] == 'a' and (pic[:-5]+'c.tif' in diff_angle)==False and (pic[:-5]+'c.tif' in miss_view)==False:
        data_names_c.remove(pic[:-5]+'c.tif')
#print(len(data_names_a))
#print(data_names_a)
#print(len(data_names_c))
#print(data_names_c)

f_1.close()
f_2.close()
#print(data_names)

for i, n in enumerate(range(len(data_names_a))):
    if i <= int(len(data_names_a)*0.75):
        shutil.copy(data_names_a[n], 'data/train_a/class_att')
    else:
        shutil.copy(data_names_a[n], 'data/test_a/class_att')
for i, n in enumerate(range(len(data_names_c))):
    if i <= int(len(data_names_c)*0.75):
        shutil.copy(data_names_c[n], 'data/train_c/class_att')
    else:
        shutil.copy(data_names_c[n], 'data/test_c/class_att')

In [5]:
data_names_a = list(glob.glob('data/Aldanella_sibirica_sp_nov/*Plate_?*_?*a.tif'))
data_names_c = list(glob.glob('data/Aldanella_sibirica_sp_nov/*Plate_?*_?*c.tif'))
f_1 = open('data/Aldanella_sibirica_sp_nov/different_angle.txt', 'rt')
f_2 = open('data/Aldanella_sibirica_sp_nov/miss_view.txt', 'rt')
diff_angle = f_1.readlines()
miss_view = f_2.readlines()
for i, pic in enumerate(diff_angle):
    diff_angle[i] = f'data/Aldanella_sibirica_sp_nov\\{pic.rstrip()}.tif'#обрезаем символы на конце строки '\n'
for i, pic in enumerate(miss_view):
    miss_view[i] = f'data/Aldanella_sibirica_sp_nov\\{pic.rstrip()}.tif'

for pic in diff_angle:
    if pic in data_names_a:
        data_names_a.remove(pic)
        if pic[:-5]+'c.tif' in data_names_c:
            data_names_c.remove(pic[:-5]+'c.tif')
for pic in diff_angle:
    if pic in data_names_c:
        data_names_c.remove(pic)

for pic in miss_view:
    if (pic[-5:-4] == 'c')==True and (pic[:-5]+'a.tif' in diff_angle)==False and (pic[:-5]+'a.tif' in miss_view)==False:
        data_names_a.remove(pic[:-5]+'a.tif')
    if pic[-5:-4] == 'a' and (pic[:-5]+'c.tif' in diff_angle)==False and (pic[:-5]+'c.tif' in miss_view)==False:
        data_names_c.remove(pic[:-5]+'c.tif')

#print(len(data_names_a))
#print(data_names_a)
#print(len(data_names_c))
#print(data_names_c)

f_1.close()
f_2.close()
#print(data_names)

for i, n in enumerate(range(len(data_names_a))):
    if i <= int(len(data_names_a)*0.75):
        shutil.copy(data_names_a[n], 'data/train_a/class_sib')
    else:
        shutil.copy(data_names_a[n], 'data/test_a/class_sib')
for i, n in enumerate(range(len(data_names_c))):
    if i <= int(len(data_names_c)*0.75):
        shutil.copy(data_names_c[n], 'data/train_c/class_sib')
    else:
        shutil.copy(data_names_c[n], 'data/test_c/class_sib')

### 1.3 Создание тензеров из изображений

In [6]:
datagen = ImageDataGenerator(rescale=1./255) # масштабируем именно значения каждого пикселя

In [7]:
#Пропускаем через свёртучнуя чать сети изображения (как "train", так и "test"), чтобы выделить массивы наиболее важных признаков

train_generator_a = datagen.flow_from_directory('data/train_a/',
                                        target_size=(150, 150),
                                        batch_size=10, #количество изображений, пропускаемых через сеть на каждой итерации
                                        color_mode='rgb',
                                        shuffle=False)
train_generator_c = datagen.flow_from_directory('data/train_c/',
                                        target_size=(150, 150),
                                        batch_size=10, #количество изображений, пропускаемых через сеть на каждой итерации
                                        color_mode='rgb',
                                        shuffle=False)

#Трансофрмацию валидационных данных не нужно делать
test_generator_a = datagen.flow_from_directory('data/test_a/',
                                              target_size=(150, 150),
                                              batch_size=10, #количество изображений, пропускаемых через сеть на каждой итерации,
                                              color_mode='rgb',
                                              shuffle=False)
test_generator_c = datagen.flow_from_directory('data/test_c/',
                                              target_size=(150, 150),
                                              batch_size=10, #количество изображений, пропускаемых через сеть на каждой итерации,
                                              color_mode='rgb',
                                              shuffle=False)

Found 59 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 18 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


## 2. Создание модели

In [8]:
inc_model_a = InceptionV3(include_top=False, weights='imagenet', pooling='avg', input_shape=((150, 150, 3)))

In [9]:
inc_model_c = ResNet50(include_top=False, weights='imagenet', pooling='avg', input_shape=((150, 150, 3)))

In [10]:
features_train_a = inc_model_a.predict(train_generator_a)
np.save(open('data/binary/train_a_binary/bn_features_exp2_train_a.npy', 'wb'), features_train_a)

features_train_c = inc_model_c.predict(train_generator_c)
np.save(open('data/binary/train_c_binary/bn_features_exp2_train_c.npy', 'wb'), features_train_c)

features_test_a = inc_model_a.predict(test_generator_a)
np.save(open('data/binary/test_a_binary/bn_features_exp2_test_a.npy', 'wb'), features_test_a)

features_test_c = inc_model_c.predict(test_generator_c)
np.save(open('data/binary/test_c_binary/bn_features_exp2_test_c.npy', 'wb'), features_test_c)

2/2 [==============================] - 0s 113ms/step


In [11]:
train_data_a = np.load(open('data/binary/train_a_binary/bn_features_exp2_train_a.npy', 'rb'))
train_labels_a = np.array([0] * 16 + [1] * 43)

train_data_c = np.load(open('data/binary/train_c_binary/bn_features_exp2_train_c.npy', 'rb'))
train_labels_c = np.array([0] * 16 + [1] * 43) 

test_data_a = np.load(open('data/binary/test_a_binary/bn_features_exp2_test_a.npy', 'rb'))
test_labels_a = np.array([0] * 5 + [1] * 13)

test_data_c = np.load(open('data/binary/test_c_binary/bn_features_exp2_test_c.npy', 'rb'))
test_labels_c = np.array([0] * 5 + [1] * 13) 

In [12]:
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data_a.shape[1:])) #Явно не создаём слой ввода (input)
                                                        #'Flatten' cжимает тензор на входе до вектора заданной формы
fc_model.add(Dense(128, activation='relu', name='dense_one')) #можно попробовать функцию активации 'selu'
fc_model.add(Dropout(0.5, name='dropout_one')) #выбрасываем сигналы от половины нейронов
fc_model.add(Dense(128, activation='relu', name='dense_two'))
fc_model.add(Dropout(0.5, name='dropout_two'))
fc_model.add(Dense(128, activation='relu', name='dense_three'))
fc_model.add(Dropout(0.5, name='dropout_three'))
fc_model.add(Dense(1, activation='sigmoid', name='output')) #использовать 'softmax' для многоклассовой классификации
#количество нейронов в выходном слое равно количеству классов (записываем максимальный номер класса)

fc_model.compile(optimizer='adam', #поиск минимума (может быть 'adam', 'sgd', 'rmsprop')
              loss='binary_crossentropy', # 'categorical_crossentropy'
              metrics=['accuracy'])

In [13]:
fc_model.fit(train_data_a, train_labels_a,
            epochs=50, batch_size=10,
            validation_data=(test_data_a, test_labels_a))

fc_model.save_weights('data/model_weights/exp2_a.hdf5')

Epoch 1/50
6/6 [==============================] - 1s 39ms/step - loss: 1.5882 - accuracy: 0.4746 - val_loss: 0.9670 - val_accuracy: 0.7222
Epoch 2/50
6/6 [==============================] - 0s 12ms/step - loss: 1.5829 - accuracy: 0.5763 - val_loss: 0.7076 - val_accuracy: 0.7222
Epoch 3/50
6/6 [==============================] - 0s 12ms/step - loss: 1.7065 - accuracy: 0.6102 - val_loss: 0.8299 - val_accuracy: 0.7222
Epoch 4/50
6/6 [==============================] - 0s 14ms/step - loss: 1.6846 - accuracy: 0.5763 - val_loss: 0.7681 - val_accuracy: 0.7222
Epoch 5/50
6/6 [==============================] - 0s 13ms/step - loss: 1.3465 - accuracy: 0.6441 - val_loss: 0.5767 - val_accuracy: 0.6667
Epoch 6/50
6/6 [==============================] - 0s 13ms/step - loss: 0.7318 - accuracy: 0.6441 - val_loss: 0.5362 - val_accuracy: 0.7222
Epoch 7/50
6/6 [==============================] - 0s 14ms/step - loss: 1.0762 - accuracy: 0.5932 - val_loss: 0.5377 - val_accuracy: 0.7222
Epoch 8/50
6/6 [===========

In [14]:
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data_a.shape[1:])) #Явно не создаём слой ввода (input)
                                                        #'Flatten' cжимает тензор на входе до вектора заданной формы
fc_model.add(Dense(128, activation='relu', name='dense_one')) #можно попробовать функцию активации 'selu'
fc_model.add(Dropout(0.5, name='dropout_one')) #выбрасываем сигналы от половины нейронов
fc_model.add(Dense(128, activation='relu', name='dense_two'))
fc_model.add(Dropout(0.5, name='dropout_two'))
fc_model.add(Dense(128, activation='relu', name='dense_three'))
fc_model.add(Dropout(0.5, name='dropout_three'))
fc_model.add(Dense(1, activation='sigmoid', name='output')) #использовать 'softmax' для многоклассовой классификации
#количество нейронов в выходном слое равно количеству классов (записываем максимальный номер класса)

fc_model.compile(optimizer='adam', #поиск минимума (может быть 'adam', 'sgd', 'rmsprop')
              loss='binary_crossentropy', # 'categorical_crossentropy'
              metrics=['accuracy'])

In [15]:
fc_model.fit(train_data_c, train_labels_c,
            epochs=50, batch_size=10,
            validation_data=(test_data_c, test_labels_c))

fc_model.save_weights('data/model_weights/exp2_c.hdf5')

Epoch 1/50
6/6 [==============================] - 1s 32ms/step - loss: 0.6065 - accuracy: 0.6610 - val_loss: 0.6076 - val_accuracy: 0.7222
Epoch 2/50
6/6 [==============================] - 0s 13ms/step - loss: 0.7123 - accuracy: 0.6610 - val_loss: 0.6262 - val_accuracy: 0.7222
Epoch 3/50
6/6 [==============================] - 0s 14ms/step - loss: 0.6492 - accuracy: 0.7119 - val_loss: 0.5969 - val_accuracy: 0.7222
Epoch 4/50
6/6 [==============================] - 0s 15ms/step - loss: 0.6772 - accuracy: 0.6441 - val_loss: 0.5998 - val_accuracy: 0.7222
Epoch 5/50
6/6 [==============================] - 0s 22ms/step - loss: 0.6026 - accuracy: 0.6949 - val_loss: 0.6026 - val_accuracy: 0.7222
Epoch 6/50
6/6 [==============================] - 0s 13ms/step - loss: 0.6190 - accuracy: 0.7288 - val_loss: 0.5933 - val_accuracy: 0.7222
Epoch 7/50
6/6 [==============================] - 0s 12ms/step - loss: 0.7196 - accuracy: 0.6441 - val_loss: 0.5933 - val_accuracy: 0.7222
Epoch 8/50
6/6 [===========

In [50]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator_a = train_datagen.flow_from_directory('data/train_a/',
                                        target_size=(150, 150),
                                        batch_size=100,
                                        color_mode='rgb',
                                        class_mode='binary',
                                        shuffle=True)
train_generator_c = train_datagen.flow_from_directory('data/train_c/',
                                        target_size=(150, 150),
                                        batch_size=100,
                                        color_mode='rgb',
                                        class_mode='binary',
                                        shuffle=True)

validation_generator_a = test_datagen.flow_from_directory('data/test_a/',
                                              target_size=(150, 150),
                                              batch_size=100,
                                              color_mode='rgb',
                                              class_mode='binary',
                                              shuffle=True)
validation_generator_c = test_datagen.flow_from_directory('data/test_c/',
                                              target_size=(150, 150),
                                              batch_size=100,
                                              color_mode='rgb',
                                              class_mode='binary',
                                              shuffle=True)

'''
pred_generator=test_datagen.flow_from_directory('data/img_val/',
                                                     target_size=(150,150),
                                                     batch_size=100,
                                                     class_mode='binary')
'''

Found 59 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 18 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


"\npred_generator=test_datagen.flow_from_directory('data/img_val/',\n                                                     target_size=(150,150),\n                                                     batch_size=100,\n                                                     class_mode='binary')\n"

In [51]:
X_train_a, y_train_a = train_generator_a.next()
X_train_c, y_train_c = train_generator_c.next()

X_validation_a, y_validation_a = validation_generator_a.next()
X_validation_c, y_validation_c = validation_generator_c.next()

In [52]:
len(X_train_a)

59

In [17]:
x = Flatten()(inc_model_a.output)
x = Dense(128, activation='relu', name='dense_a_one')(x)
x = Dropout(0.5, name='dropout_a_one')(x)
x = Dense(128, activation='relu', name='dense_a_two')(x)
x = Dropout(0.5, name='dropout_a_two')(x)
x = Dense(128, activation='relu', name='dense_a_three')(x)
x = Dropout(0.5, name='dropout_a_three')(x)
#top_model=Dense(1, activation='sigmoid', name='output')(x)
model_a = Model(inputs=inc_model_a.input, outputs=x)

In [18]:
x = Flatten()(inc_model_c.output)
x = Dense(128, activation='relu', name='dense_c_one')(x)
x = Dropout(0.5, name='dropout_c_one')(x)
x = Dense(128, activation='relu', name='dense_c_two')(x)
x = Dropout(0.5, name='dropout_c_two')(x)
x = Dense(128, activation='relu', name='dense_c_three')(x)
x = Dropout(0.5, name='dropout_c_three')(x)
#top_model=Dense(1, activation='sigmoid', name='output')(x)
model_c = Model(inputs=inc_model_c.input, outputs=x)

In [19]:
weights_filename='data/model_weights/exp2_a.hdf5'
model_a.load_weights(weights_filename, by_name=True)

weights_filename='data/model_weights/exp2_c.hdf5'
model_c.load_weights(weights_filename, by_name=True)

In [106]:
combinedInput = Concatenate()([model_a.output, model_c.output])
x = Dense(256, activation='relu')(combinedInput)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
top = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[model_a.input, model_c.input], outputs=top)

In [107]:
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [111]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=1e-4, momentum=0.9),
                #optimizer='adam',
              #optimizer=Adam(learning_rate=1e-4, decay=1e-4 / 200),
              metrics=['accuracy'])

In [112]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [113]:
model.fit([X_train_a, X_train_c], train_labels_a,
            validation_data=([X_validation_a, X_validation_c], y_validation_a),
            epochs=200, batch_size=10,
            callbacks=[early_stopping])

Epoch 1/200
6/6 [==============================] - 10s 427ms/step - loss: 0.2610 - accuracy: 0.7458 - val_loss: 1.1433 - val_accuracy: 0.7222
Epoch 2/200
6/6 [==============================] - 1s 122ms/step - loss: 0.2016 - accuracy: 0.7627 - val_loss: 1.1402 - val_accuracy: 0.7222
Epoch 3/200
6/6 [==============================] - 1s 122ms/step - loss: 0.2123 - accuracy: 0.7458 - val_loss: 1.1436 - val_accuracy: 0.7222
Epoch 4/200
6/6 [==============================] - 1s 123ms/step - loss: 0.2220 - accuracy: 0.7288 - val_loss: 1.1525 - val_accuracy: 0.7222
Epoch 5/200
6/6 [==============================] - 1s 123ms/step - loss: 0.2220 - accuracy: 0.7458 - val_loss: 1.1592 - val_accuracy: 0.7222
Epoch 6/200
6/6 [==============================] - 1s 123ms/step - loss: 0.2229 - accuracy: 0.7797 - val_loss: 1.1683 - val_accuracy: 0.7222
Epoch 7/200
6/6 [==============================] - 1s 123ms/step - loss: 0.2473 - accuracy: 0.7458 - val_loss: 1.1805 - val_accuracy: 0.7222
Epoch 8/200


In [114]:
result = model.evaluate([X_validation_a, X_validation_c], y_validation_a)
dict(zip(fc_model.metrics_names, result))

1/1 [==============================] - 0s 217ms/step - loss: 1.2141 - accuracy: 0.7222


{'loss': 1.2141032218933105, 'accuracy': 0.7222222089767456}